In [1]:
%load_ext autoreload
%autoreload 2
import penzai
from penzai import pz
pz.ts.register_as_default()
pz.ts.register_autovisualize_magic()
pz.enable_interactive_context()

In [2]:
filename = "models/phi-3-16.gguf"
from micrlhf.llama import LlamaTransformer
llama = LlamaTransformer.from_pretrained(filename, device_map="auto")

In [21]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("microsoft/Phi-3-mini-4k-instruct")
tokenizer.padding_side = "right"

/home/neverix/.cache/pypoetry/virtualenvs/micrlhf-progress-a058ydGG-py3.12/lib/python3.12/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [22]:
!mkdir -p data
!wget -c 'https://people.eecs.berkeley.edu/~hendrycks/data.tar' -O data/data.tar
import pandas as pd
import tarfile


combined_prompt = """<|user|>
{}
Choices:
(A) {}
(B) {}
(C) {}
(D) {}<|end|>
<|assistant|>
Answer: ({}"""
dataset = []
with tarfile.open("data/data.tar") as data:
    for m in data.getmembers():
        if not m.name.startswith("data/val"):
            continue
        if not m.name.endswith(".csv"):
            continue
        df = pd.read_csv(data.extractfile(m))
        for _, r in df.iterrows():
            dataset.append(combined_prompt.format(*r.tolist()))

/home/neverix/.pyenv/versions/3.12.3/lib/python3.12/pty.py:95: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


--2024-05-19 22:48:06--  https://people.eecs.berkeley.edu/~hendrycks/data.tar
Resolving people.eecs.berkeley.edu (people.eecs.berkeley.edu)... 128.32.244.190
Connecting to people.eecs.berkeley.edu (people.eecs.berkeley.edu)|128.32.244.190|:443... connected.
HTTP request sent, awaiting response... 416 Requested Range Not Satisfiable

    The file is already fully retrieved; nothing to do.



In [23]:
from micrlhf.sampling import jit_wrapper


llama_call = jit_wrapper.Jitted(llama)

In [30]:
from more_itertools import chunked
from tqdm.auto import tqdm
from micrlhf.sampling import jnp
import jax


tokenizer.padding_side = "right"
accuracies = []
batch_size = 128
for batch in chunked(tqdm(dataset), batch_size):
    og_batch_size = len(batch)
    batch = batch + [""] * (batch_size - len(batch))
    tokens = tokenizer.batch_encode_plus(batch,
                                        return_tensors="np",
                                        padding="max_length",
                                        truncation=True,
                                        max_length=256,
                                        return_attention_mask=True)
    token_array = jnp.asarray(tokens["input_ids"])
    token_array = jax.device_put(token_array, jax.sharding.NamedSharding(llama.mesh, jax.sharding.PartitionSpec("dp", "sp")))
    token_array = pz.nx.wrap(token_array, "batch", "seq").untag("batch").tag("batch")
    inputs = llama.inputs.from_basic_segments(token_array)
    logits = llama_call(inputs)
    probs = pz.nx.nmap(jax.nn.softmax)(logits.untag("vocabulary")).tag("vocabulary")
    mask = pz.nx.wrap(jnp.asarray(tokens["attention_mask"]), "batch", "seq")
    probs = pz.nx.nmap(lambda p, m, i: p[m.sum() - 2, i[m.sum() - 1]])(
        probs.untag("seq", "vocabulary"), mask.untag("seq"), token_array.untag("seq"))
    accuracies.append(float(probs.data_array[:og_batch_size].mean()))
sum(accuracies) / len(accuracies)

  0%|          | 0/1474 [00:00<?, ?it/s]

0.6920572916666666